In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import spacy
import random
import json
from spacy.training.example import Example
from spacy.util import Adam
from spacy.scorer import Scorer
from spacy import displacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [4]:
def evaluate(ner_model, samples):
    scorer = Scorer(ner_model)
    example = []
    for sample in samples:
        pred = ner_model(sample[0])
        #print(pred, sample[1]['entities'])
        temp_ex = Example.from_dict(pred, {'entities': sample[1]['entities']})
        example.append(temp_ex)
    scores = scorer.score(example)

    return scores

In [5]:
labeled_data = []
with open("/content/drive/MyDrive/marked_up_dataset_1.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)

In [6]:
for entry in labeled_data:
    if entry['Entities'] == []:
        continue
    elif entry['Entities'][0][2] == 'techologies':
        for j in range(len(entry['Entities'])):
            entry['Entities'][j][2] = 'technologies'



In [7]:
i = 0
for entry in labeled_data:
    if entry['Entities'] == []:
        continue
    elif entry['Entities'][0][2] == 'techologies':
        i = i + 1

i

0

In [8]:
TRAIN_DATA = []
TEST_DATA = []
VAL_DATA = []
for entry in labeled_data:
    entities = []
    for e in entry['Entities']:
        entities.append((e[0], e[1],e[2]))
    spacy_entry = (entry['Data'], {"entities": entities})
    if entry['Label'] == 'train':
        TRAIN_DATA.append(spacy_entry)
    elif entry['Label'] == 'test':
        TEST_DATA.append(spacy_entry)
    elif entry['Label'] == 'val':
        VAL_DATA.append(spacy_entry)

In [9]:
print(f'train_len = {len(TRAIN_DATA)}, --- test_len = {len(TEST_DATA)}, --- val_len = {len(VAL_DATA)}')

train_len = 450, --- test_len = 113, --- val_len = 0


In [10]:
nlp = spacy.blank("ru")
ner = nlp.create_pipe("ner")

In [11]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [12]:
nlp.add_pipe("ner")
ner.add_label("technologies")

1

In [13]:
optimizer = nlp.resume_training()

In [14]:
nlp.begin_training()

with nlp.disable_pipes(*unaffected_pipes):

    for itn in range(10):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 2))
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3, sgd=optimizer)

        print(f'train loss: {losses}')
        val_loss = evaluate(nlp, VAL_DATA)
        print(val_loss['ents_per_type'])

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Разработчик / Ведущий разработчик Java на долгосро..." with entities "[(34, 38, 'technologies'), (498, 510, 'technologie...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Небольшая компания ищет программиста встраиваемых ..." with entities "[(810, 812, 'technologies'), (823, 832, 'technolog...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

train loss: {'ner': 5916.830063222443}
None
train loss: {'ner': 2952.61293981295}
None
train loss: {'ner': 2604.6523198061814}
None
train loss: {'ner': 2342.7009312118985}
None
train loss: {'ner': 2190.177913539227}
None
train loss: {'ner': 2053.5196820226465}
None
train loss: {'ner': 1860.555875591802}
None
train loss: {'ner': 1812.6431021039518}
None
train loss: {'ner': 1753.3811978116642}
None
train loss: {'ner': 1620.7319722752413}
None


In [15]:
results = evaluate(nlp, TEST_DATA)
results['ents_per_type']

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Липтсофт - генеральный it-партнер банка &quot;Санк..." with entities "[(264, 269, 'technologies'), (304, 308, 'technolog...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Привет! Кошелёк — мобильное приложение, которое за..." with entities "[(638, 642, 'technologies'), (657, 659, 'technolog...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

{'technologies': {'p': 0.7313054499366286,
  'r': 0.7079754601226994,
  'f': 0.7194513715710723}}

In [24]:
example = TEST_DATA[10][0]

In [28]:
example

'Липтсофт - генеральный it-партнер банка &quot;Санкт-Петербург&quot;. Мы автономная компания с быстрыми процессами и уютной атмосферой. Есть возможность удаленной работы. Задачи:- Разработка нативного iOS приложения для юридических и физических лиц Опыт и навыки:- Swift от 3 лет коммерческой разработки- REST, TCP, HTTPS(handshake, certificate, MITM understanding)- IB, Autolayout, Верстка из кода,- RxSwift- MVC, MVVM- ООП, SOLID, Design Patterns.- Multithreading (GCD, OperationQueue)- Git Мы предлагаем:- Высокий уровень зп. Отталкиваемся от уровня скилов каждого кандидата.- Маленькая команда опытных профессионалов, плоская иерархия и никакой бюрократии- Новый офис в БЦ класса «А» («Санкт-Петербург Плаза», м. Новочеркасская)- Не боимся экспериментов с любыми доступными технологиями и решениями- Оплачиваем участие в конференциях и техническую литературу- ДМС (со стоматологией), курсы английского в офисе, абонемент в спортзал- Отсутствие дресс-кода, гибкое начало дня с 9:00 до 11:00'

In [25]:
doc = nlp(example)

In [27]:
doc.ents

(iOS,
 Swift,
 REST,
 TCP,
 IB,
 Autolayout,
 MVVM- ООП,
 SOLID,
 Design Patterns.-,
 Git)